In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sagemaker
import boto3
import matplotlib.pyplot as plt 
pd.set_option('display.max_columns',None) # To display all columns
pd.set_option('display.max_rows',None) # To display all columns
client = boto3.client('s3')
import feather
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from functools import reduce
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
df1 = pd.read_feather('FinalData.ftr')
df2 = pd.read_feather('Dataframe80.ftr')
dfu = pd.read_feather('s3://haseebdata/Dataset/Users.ftr')
dfq = pd.read_feather('s3://haseebdata/Dataset/dfq.ftr')
Repeat=pd.read_feather('RepeatedQuestion.ftr')

In [3]:
ix = df2[df2.correct.eq(1)].groupby(['user_iD', 'question_id'])['elapsed_time'].idxmax()
df3= df2.loc[ix].reset_index()
df3[0:5]

,index,user_iD,question_id,user_answer,correct_answer,correct,elapsed_time,solving_id,explanation_id,bundle_id,timestamp,deployed_at
0,1818,50,3,b,b,1.0,23.0,1729,3,3,1560337981670,2019-05-17 11:40:22.784
1,2481,50,4,b,b,1.0,29.0,2109,4,4,1560769502979,2019-05-17 11:40:25.357
2,2149,50,5,c,c,1.0,24.0,1883,5,5,1560513025097,2019-05-17 11:40:28.439
3,1838,50,7,d,d,1.0,23.0,1747,7,7,1560339686648,2019-05-17 11:40:33.992
4,8355,50,29,d,d,1.0,19.0,3905,29,29,1571227991581,2019-09-17 02:45:04.653


In [8]:
del df3['index']

In [9]:
df3.to_feather('df.ftr')

In [5]:
df_i = df3.groupby(['user_iD', 'question_id']).size().reset_index(name='counts')
df_i.head()

,user_iD,question_id,counts
0,50,3,1
1,50,4,1
2,50,5,1
3,50,7,1
4,50,29,1


In [6]:
df_total_questions_attempted = df3.groupby(['user_iD'], as_index=False , sort=False)['question_id'].count()
df_total_questions_attempted = df_total_questions_attempted.rename(columns={'question_id': 'Unique Attempted'})

df_correct = df3[df3['correct']==1].groupby(['user_iD'], as_index=False, sort=False)['question_id'].count()

df_correct = df_correct.rename(columns={'question_id': 'Corrected'})

#df_incorrect = df3[df3['correct']==0].groupby(['user_iD'], as_index=False, sort=False)['question_id'].count()

#df_incorrect = df_incorrect.rename(columns={'question_id': 'Incorrected'})

dfs = [df_total_questions_attempted, df_correct]
df4 = reduce(lambda left,right: pd.merge(left,right,on='user_iD'), dfs)

df4['Percentage'] = (df4['Corrected'] / df4['Unique Attempted']) *100

#df_final['elapsed_time'] = df2['elapsed_time'] * 1000

#df2['elapsed_time']= df2.elapsed_time.div(1000)   #DIVDING THE ELAPSED TIME by 1000 to further check the mean elapsed time by each user
#Elapsed_time= df2.groupby('user_iD' , sort=False)['elapsed_time'].median()

#dfs = [df_final, Elapsed_time]
#df4 = reduce(lambda left,right: pd.merge(left,right,on='user_iD'), dfs)

In [7]:
df4.head()

,user_iD,Unique Attempted,Corrected,Percentage
0,50,4570,4570,100.0
1,58,5191,5191,100.0
2,98,4731,4731,100.0
3,99,39,39,100.0
4,103,6,6,100.0
